In [ ]:
import glob
import os
import pickle
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    X = X[:20*sample_rate]
    median = np.median(np.abs(X))
    X = X / median if median > 0 else X
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def getLabel(fileName):
    return 1 if ('/snore_' in fileName) or ('/positive/' in fileName) else 0

def parse_audio_files(rootDir,file_ext='*.wav',limit=None):
    features, labels = np.empty((0,193)), np.empty(0)
    count = 0
    for fn in glob.iglob(os.path.join(rootDir, '**', file_ext), recursive=True):
        print(fn)
        mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
        ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        features = np.vstack([features,ext_features])
        labels = np.append(labels, getLabel(fn))
        count += 1
        if limit != None and count > limit:
            break
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
#extract_feature('/home/jan/projects/sleeplab/snoring/src/main/samples/verified/snore_clear/cerulean47@gmail.com---rec.1473848857853.wav')

In [ ]:
def extractFeatures(rootDir, name, limit=None):
    
    features, labels = parse_audio_files(rootDir,limit=limit)
    labels = one_hot_encode(labels)
    
    pickleFile = open(name, 'wb')
    pickle.dump(features, pickleFile, pickle.HIGHEST_PROTOCOL)
    pickle.dump(labels, pickleFile, pickle.HIGHEST_PROTOCOL)
    pickleFile.close()
    
def loadFeatures(name):
    pickleFile = open(name, 'rb')
    features = pickle.load(pickleFile)
    labels = pickle.load(pickleFile)
    pickleFile.close()
    return features, labels

In [ ]:
def printLearningResults(testy, testyp):
    testy = testy.astype(dtype=bool)
    testyp = testyp.astype(dtype=bool)
    print("Accuracy ", np.sum(testy == testyp) / testy.size)
    tp = np.sum(testy & testyp)
    print("True positives ", tp)
    fp = np.sum(~testy & testyp)
    print("False positives ", fp)
    tn = np.sum(~testy & ~testyp)
    print("True negatives ", tn)
    fn = np.sum(testy & ~testyp)
    print("False negatives ", fn)
    se = tp / (tp+fn)
    print("Sensitivity ",se)
    sp = tn / (tn+fp)
    print("Specificity ",sp)
    print("Youden ", se+sp-1)

In [ ]:
#extractFeatures('../sleeplab/snoring/src/main/samples/verified','snore_verified_20s_norm.p')
#extractFeatures('../sleeplab/snoring/src/main/samples/unbiased','snore_unbiased_20s_norm.p')
#extractFeatures('../sleeplab/snoring/src/main/samples/original','snore_original_20s_norm.p')

In [ ]:
features, labels = loadFeatures('snore_verified_20s_norm.p')

In [ ]:
train_test_split = np.random.rand(len(features)) < 0.70
train_x = features[train_test_split]
train_y = labels[train_test_split]
test_x = features[~train_test_split]
test_y = labels[~train_test_split]

#### Training Neural Network with TensorFlow

In [ ]:
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
n_dim = features.shape[1]
n_classes = 2
n_hidden_units_one = 1000
n_hidden_units_two = 1000
sd = 1 / np.sqrt(n_dim)
learning_rate = 0.01

In [ ]:
X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,n_classes])

W_1 = tf.Variable(tf.random_normal([n_dim,n_hidden_units_one], mean = 0, stddev=sd))
b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean = 0, stddev=sd))
h_1 = tf.nn.tanh(tf.matmul(X,W_1) + b_1)


W_2 = tf.Variable(tf.random_normal([n_hidden_units_one,n_hidden_units_two], mean = 0, stddev=sd))
b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean = 0, stddev=sd))
h_2 = tf.nn.sigmoid(tf.matmul(h_1,W_2) + b_2)


W = tf.Variable(tf.random_normal([n_hidden_units_two,n_classes], mean = 0, stddev=sd))
b = tf.Variable(tf.random_normal([n_classes], mean = 0, stddev=sd))
y_ = tf.nn.softmax(tf.matmul(h_2,W) + b)

init = tf.initialize_all_variables()

In [ ]:
cost_function = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(y_), reduction_indices=[1])) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
cost_history = np.empty(shape=[1],dtype=float)
sess = tf.Session()
sess.run(init)

In [ ]:
for epoch in range(5000):           
    _,cost = sess.run([optimizer,cost_function],feed_dict={X:train_x,Y:train_y})
    cost_history = np.append(cost_history,cost)
    if epoch % 1000 == 0:
        print(epoch,cost)

In [ ]:
plt.plot(cost_history[1:])

In [ ]:
y_pred = sess.run(tf.argmax(y_,1),feed_dict={X: test_x})
y_true = sess.run(tf.argmax(test_y,1))
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
printLearningResults(y_true, y_pred)

In [ ]:
f2, l2 = loadFeatures('snore_unbiased_20s_norm.p')
y_pred = sess.run(tf.argmax(y_,1),feed_dict={X: f2})
y_true = sess.run(tf.argmax(l2,1))
printLearningResults(y_true, y_pred)

In [ ]:
f2, l2 = loadFeatures('snore_original_20s_norm.p')
y_pred = sess.run(tf.argmax(y_,1),feed_dict={X: f2})
y_true = sess.run(tf.argmax(l2,1))
printLearningResults(y_true, y_pred)

In [ ]:
sess.close()